In [2]:
import sys, os
import numpy as np
import h5py
import math
import random
import neptune
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow_addons as tfa
import tensorflow.keras.layers as nn

from tqdm import tqdm
from PIL import Image
from tensorflow import keras, einsum
from tensorflow_probability import layers as tfpl
from tensorflow_probability import distributions as tfd
from tensorflow.keras import Model, Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Layer, Input, MaxPooling2D, Conv2D, Conv2DTranspose, Flatten, Dropout, Dense, BatchNormalization, Activation, Reshape
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from tensorflow_probability.python.distributions import kl_divergence
from einops import rearrange
from einops.layers.tensorflow import Rearrange

from functools import partial
from inspect import isfunction

In [3]:
%%capture
gpus = tf.config.list_physical_devices('GPU');
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit = 20000)]);
        logical_gpus = tf.config.experimental.list_logical_devices('GPU');
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

2023-10-18 12:02:41.785125: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-18 12:02:41.815896: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-18 12:02:41.816185: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
hf_train = h5py.File('/data/HSC/HSC_v6/step2A/64x64/5x64x64_training_z_less_than_2.hdf5', 'r')
hf_test = h5py.File('/data/HSC/HSC_v6/step2A/64x64/5x64x64_testing_z_less_than_2.hdf5', 'r')
hf_validation = h5py.File('/data/HSC/HSC_v6/step2A/64x64/5x64x64_validation_z_less_than_2.hdf5', 'r')
x_train = np.asarray(np.transpose(hf_train['image'][0 : ],(0, 2, 3, 1)))
x_test = np.asarray(np.transpose(hf_test['image'][0 : ],(0, 2, 3, 1)))
x_validation = np.asarray(np.transpose(hf_validation['image'][0 : ], (0, 2, 3, 1)))
y_train = np.asarray(hf_train['specz_redshift'][0 : ])[..., None]
y_test = np.asarray(hf_test['specz_redshift'][0 : ])[..., None]
y_validation = np.asarray(hf_validation['specz_redshift'][0 : ])[..., None]
hf_train.close()
hf_test.close()
hf_validation.close()

In [6]:
x_train.shape

(194510, 64, 64, 5)

In [8]:
dir_name = "/data3/Billy/data/HSC/HSC_v6/step2A/64x64/"
os.makedirs(dir_name, exist_ok=True)

In [16]:
n_files = x_train.shape[0] // 106
n_files

1835

In [9]:
n_files = x_train.shape[0] // 106  # the number of smaller arrays
split_arrays = np.array_split(x_train, n_files)

# Save each smaller array into a file
for i, small_array in enumerate(split_arrays):
    filename = f"/data3/Billy/data/HSC/HSC_v6/step2A/64x64/5x64x64_training_z_less_than_2_batch_{i}.npy"
    np.save(filename, small_array)

print("All files saved!")

All files saved!


In [12]:
n_files = y_train.shape[0] // 106  # the number of smaller arrays
split_arrays = np.array_split(y_train, n_files)

# Save each smaller array into a file
for i, small_array in tqdm(enumerate(split_arrays)):
    filename = f"/data3/Billy/data/HSC/HSC_v6/step2A/64x64/5x64x64_training_z_less_than_2_batch_{i}_redshifts.npy"
    np.save(filename, small_array)

print("All files saved!")

1835it [00:00, 24961.80it/s]

All files saved!


In [10]:
n_files = x_test.shape[0] // 106  # the number of smaller arrays
split_arrays = np.array_split(x_test, n_files)

# Save each smaller array into a file
for i, small_array in tqdm(enumerate(split_arrays)):
    filename = f"/data3/Billy/data/HSC/HSC_v6/step2A/64x64/5x64x64_testing_z_less_than_2_batch_{i}.npy"
    np.save(filename, small_array)

print("All files saved!")

366it [00:12, 30.47it/s]

All files saved!


In [13]:
n_files = y_test.shape[0] // 106  # the number of smaller arrays
split_arrays = np.array_split(y_test, n_files)

# Save each smaller array into a file
for i, small_array in tqdm(enumerate(split_arrays)):
    filename = f"/data3/Billy/data/HSC/HSC_v6/step2A/64x64/5x64x64_testing_z_less_than_2_batch_{i}_redshifts.npy"
    np.save(filename, small_array)

print("All files saved!")

366it [00:00, 13109.33it/s]

All files saved!


In [11]:
n_files = x_validation.shape[0] // 106  # the number of smaller arrays
split_arrays = np.array_split(x_validation, n_files)

# Save each smaller array into a file
for i, small_array in tqdm(enumerate(split_arrays)):
    filename = f"/data3/Billy/data/HSC/HSC_v6/step2A/64x64/5x64x64_validation_z_less_than_2_batch_{i}.npy"
    np.save(filename, small_array)

print("All files saved!")

367it [00:11, 30.68it/s]

All files saved!


In [14]:
n_files = y_validation.shape[0] // 106  # the number of smaller arrays
split_arrays = np.array_split(y_validation, n_files)

# Save each smaller array into a file
for i, small_array in tqdm(enumerate(split_arrays)):
    filename = f"/data3/Billy/data/HSC/HSC_v6/step2A/64x64/5x64x64_validation_z_less_than_2_batch_{i}_redshifts.npy"
    np.save(filename, small_array)

print("All files saved!")

367it [00:00, 12540.81it/s]

All files saved!
